In [ ]:
import numpy as np
from scipy.stats import norm

### 13.2.3 Delta and Gamma in Practice: Delta and Gamma by Strike

This coding example shows how to calculate the delta and gamma of European call and put options in the Black-Scholes model:

In [ ]:
s0, K, r, T, sigma = 100, 100, 0.05, 1, 0.2

d1 = (np.log(s0/K) + (r + 0.5*sigma**2) * T) / (sigma * np.sqrt(T))
delta_c = norm.cdf(d1)
delta_p = -norm.cdf(-d1)
print(delta_c, delta_p)

0.6368306511756191 -0.3631693488243809


### 13.2.5 Theta in Practice: How Does Theta Change by Option Expiry?

In this coding example, we detail how to calculate the theta of a European call and put option using equations (13.4) and (13.5):

In [ ]:
s0, K, r, T, sigma = 100, 100, 0.05, 1, 0.2

d1 = (np.log(s0/K) + (r + 0.5*sigma**2) * T) / (sigma * np.sqrt(T))
d2 = d1 - sigma * np.sqrt(T)
theta_c = -s0*norm.pdf(d1)*sigma / (2*np.sqrt(T)) - r*K*np.exp(-r*T)*norm.cdf(d2)
theta_p = -s0*norm.pdf(d1)*sigma / (2*np.sqrt(T)) + r*K*np.exp(-r*T)*norm.cdf(-d2)
print(theta_c, theta_p)

-6.414027546438197 -1.657880423934626


### 13.6 ESTIMATION OF GREEKS VIA FINITE DIFFERENCES

In this coding example we show how finite difference can be used to compute first-order Greeks for a lookback option, where the lookback option itself is valued using the simulation techniques discussed in chapter 12:

In [ ]:
def bs_simulate(s0, t, r, sigma, n, N):
    dt = 1 / n
    dwt = np.random.normal(0, np.sqrt(dt), (N, int(n * t)))
    s = np.zeros((N, dwt.shape[1] + 1))
    s[:, 0] = s0
    for i in range(1, s.shape[1]):
        s[:, i] = s[:, i-1] + s[:, i-1] * r * \
            dt + s[:, i-1] * sigma * dwt[:, i-1]
    return s

def lookback_price(k, t, is_call, paths):
    s_m = np.max(paths, axis=1) if is_call else np.min(paths, axis=1)
    return np.fmax(s_m - k, 0) if is_call else np.fmax(k - s_m, 0)

def c(s0, sigma, t, r, k, n, N):
    np.random.seed(42)        # ensure reproducibility
    paths = bs_simulate(s0, t, r, sigma, n, N)
    look_prc = np.exp(-r * t) * np.mean(lookback_price(k, t, True, paths))
    return look_prc


# example
s0 = 125        # inital asset price
t = 1           # maturity in years
r = 0.05        # annual risk free rate
sigma = 0.25    # annual volatility
n = 252         # number of steps in unit of t
N = 100000      # number of random draws
k = 125         # at-the money option
epsilon = 1e-6  # different epsilon gets different gamma

delta = (c(s0+epsilon, sigma, t, r, k, n, N) - c(s0-epsilon, sigma, t, r, k, n, N)) / (2*epsilon)
vega = (c(s0, sigma+epsilon, t, r, k, n, N) - c(s0, sigma-epsilon, t, r, k, n, N)) / (2*epsilon)
print(delta, vega)

1.1601663931770645 105.75111389421465


### 13.7.1 Smile Adjusted Greeks in Practice: USDBRL Options

In this coding example we calculate a smile adjusted delta under the Heston Model via finite differences.  To accomplish this, we shift the underlying asset as is standard in delta calculations.  However, we also shift the initial level of volatility, $\sigma_0$, as the processes are correlated and therefore a change in the asset implies a certain expected change to the volatility or variance process.  The following code details our implementation of smile-adjusted deltas in the Heston model:


In [ ]:
def heston_simulate(s0, t, r, nu0, kappa, theta, xi, rho, n, N):
    dt = 1 / n
    dwt = np.random.multivariate_normal(
        mean=(0, 0),
        cov=[[dt, rho*dt], [rho*dt, dt]],
        size=(N, int(n * t)))
    s, nu = np.zeros((N, dwt.shape[1] + 1)), np.zeros((N, dwt.shape[1] + 1))
    s[:, 0], nu[:, 0] = s0, nu0
    for i in range(1, s.shape[1]):
        s[:, i] = s[:, i-1] + s[:, i-1] * r * dt \
            + s[:, i-1] * nu[:, i-1]**0.5 * dwt[:, i-1, 0]
        nu[:, i] = nu[:, i-1] + kappa*(theta-nu[:, i-1]) \
            + xi*nu[:, i-1]**0.5*dwt[:, i-1, 1]
    s = s
    nu = nu
    return s, nu

def euro_payoffs(k, t, is_call, paths):
    st = paths[:, -1]
    return np.fmax(st - k, 0) if is_call else np.fmax(k - st, 0)

def c(s0, nu0, theta, t, r, kappa, xi, rho, n, N, k):
    np.random.seed(42)        # ensure reproducibility
    pathsS0, pathsNu = heston_simulate(s0, t, r, nu0, kappa, theta, xi, rho, n, N)
    euro_prc = np.exp(-r * t) * np.mean(euro_payoffs(k, t, True, pathsS0))
    return euro_prc


# example
s0 = 125
t = 1
r = 0.05
nu0 = 0.25**2
n = 252
N = 10000
xi = 0.3
rho = -0.5
kappa = 1
theta = nu0
k = 125     # at-the-money option
epsilon = 1e-6

dnu0_delta = (xi * rho * epsilon) / s0
smile_delta = (c(s0+epsilon, nu0+dnu0_delta, theta, t, r, kappa, xi, rho, n, N, k) - c(s0-epsilon, nu0-dnu0_delta, theta, t, r, kappa, xi, rho, n, N, k)) / (2*epsilon)
print(smile_delta)

0.6396870215752415
